# Project #2
## (your name here)

#### CS167: Machine Learning, Fall 2025

## __Put the Model in GPU mode__

We want to accelerate the training process using graphical processing unit (GPU). You need to enable it (click Settings --> Accelerator--> GPU T4 x2)

In [ ]:
import torch
# check GPU (Kaggle will show "cuda" if GPU enabled in the notebook settings)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
import torch
import numpy as np
import random

# Set seeds for reproducibility
seed = 42  # you can choose any integer
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# If using CUDA:
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if using multi-GPU

In [ ]:
# ============================================
# Step 1: imports (Kaggle version)
# ============================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import time
import os
# ===========================================

In [ ]:
# ============================================
# Step 2: Load the Data (Kaggle version)
# ============================================

base_dir   = "/kaggle/input/chest-xray"      # <-- change this name to match your Kaggle dataset
train_dir  = os.path.join(base_dir, "chest_xray/train")
test_dir   = os.path.join(base_dir, "chest_xray/test")

# Use grayscale (since X-rays are single channel) and resize to AlexNet input
transform = transforms.Compose([
    #transforms.Resize((227, 227)), ### AlexNet needs images that are 227x227
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # normalize single channel (mean,std)
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset  = datasets.ImageFolder(test_dir,  transform=transform)

dataset_labels = train_dataset.classes
number_of_classes = len(dataset_labels)
print("Classes:", dataset_labels)
print(f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}")

In [ ]:
# ============================================
# Step 3: Create your MLP Network (with pooling)
# ============================================
import torch
from torch import nn

class SimpleMLPv2(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.network_layers = nn.Sequential(
            # Input: (batch, 1, 200, 200)

            nn.MaxPool2d(kernel_size=2),   # Output: (batch, 1, 100, 100)

            nn.Flatten(),                   # Output: (batch, 10000)

            nn.Linear(100 * 100, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.network_layers(x)


In [ ]:
# ============================================
# Step 4: training / testing loops 
# ============================================
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    running_loss = 0.0
    correct = 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # forward + loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # backward + update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    avg_loss = running_loss / len(dataloader)
    accuracy = correct / size
    return avg_loss, accuracy

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    running_loss = 0.0
    correct = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)

            running_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            all_preds.append(pred.argmax(1).cpu())
            all_labels.append(y.cpu())

    avg_loss = running_loss / len(dataloader)
    accuracy = correct / size

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    conf_matrix = confusion_matrix(all_labels, all_preds)

    return avg_loss, accuracy, conf_matrix

In [ ]:
# ============================================
# Step 5: Load the Data, Train the Model, Calculate the Results
# ============================================
mlp_model = SimpleMLPv2(number_of_classes)
mlp_model.to(device)
print(mlp_model)

learning_rate   = 1e-4
batch_size_val  = 32
epochs          = 10
loss_fn         = nn.CrossEntropyLoss()
optimizer       = optim.Adam(mlp_model.parameters(), lr=learning_rate)
softmax         = nn.Softmax(dim=1)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size_val,
    shuffle=True,
    num_workers=2,        # Kaggle: use workers to speed up loading
    pin_memory=True if device == "cuda" else False
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size_val,
    shuffle=False,
    num_workers=2,
    pin_memory=True if device == "cuda" else False
)

train_losses = []
test_losses  = []
train_accuracies = []
test_accuracies  = []

start_time = time.time()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    avg_train_loss, train_accuracy = train_loop(train_dataloader, mlp_model, loss_fn, optimizer)
    avg_test_loss, test_accuracy, conf_matrix_test = test_loop(test_dataloader, mlp_model, loss_fn)

    train_losses.append(avg_train_loss)
    test_losses.append(avg_test_loss)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

    print(f"Train loss: {avg_train_loss:.4f}, Train acc: {train_accuracy:.4f}")
    print(f"Test  loss: {avg_test_loss:.4f}, Test  acc: {test_accuracy:.4f}")

print("model has been fine-tuned!")
total_time_sec = time.time() - start_time
print("Total fine-tuning time: %.3f sec" % total_time_sec)
print("Total fine-tuning time: %.3f hrs" % (total_time_sec / 3600.0))


In [ ]:
# visualizing the accuracy curves

plt.plot(range(1,epochs+1), train_accuracies)
plt.plot(range(1,epochs+1), test_accuracies)
plt.title('Model accuracies after each epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# show confusion matrix for final epoch
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_test, display_labels=dataset_labels)
disp.plot(xticks_rotation=45,cmap="Blues")
plt.tight_layout()
plt.show()